# Probability Density Functions

二项分布和泊松分布是离散分布，这意味着结果必须是不同的或独立的元素，例如命中和未命中的整数数量，或进球数。在离散分布中，每个结果都与一个概率质量相关联。

指数分布和正态分布是连续分布，这意味着结果可能出现在可能值范围内的任意一点。在连续分布中，每个结果都与一个概率密度相关联。概率密度是一个抽象概念，许多人最初会觉得难以理解，但我们将循序渐进地学习。作为第一步，让我们再次思考如何比较分布。

In [ ]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/thinkstats.py")

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from thinkstats import decorate

## 6.1 Comparing Distributions

在上一章中，当我们比较离散分布时，我们使用条形图来展示它们的概率质量函数（PMFs）。当我们比较连续分布时，我们使用线图来展示它们的累积分布函数（CDFs）。

对于离散分布，我们同样可以使用累积分布函数。例如，这是一个参数为 lam=2.2 的泊松分布的概率质量函数，该分布能很好地模拟全国家庭成长调查数据中的家庭规模分布情况。

我们可以使用 read_fem_resp 来读取受访者数据文件。

In [ ]:
from nsfg import read_fem_resp

resp = read_fem_resp()

接下来，我们将筛选出 25 岁及以上人群的家庭规模数据。

In [ ]:
older = resp.query("age >= 25")
num_family = older["numfmhh"]

并创建一个 Pmf 来表示这些回答的分布情况。

In [ ]:
from empiricaldist import Pmf

pmf_family = Pmf.from_seq(num_family, name="data")

这是另一个具有相同均值的泊松分布 Pmf 。

In [ ]:
from thinkstats import poisson_pmf

lam = 2.2
ks = np.arange(11)
ps = poisson_pmf(ks, lam)

pmf_poisson = Pmf(ps, ks, name="Poisson model")

以下是将数据分布与泊松模型进行比较的结果。

In [ ]:
from thinkstats import two_bar_plots

two_bar_plots(pmf_family, pmf_poisson)
decorate(xlabel="Number of family members")

通过比较概率质量函数，我们可以看到模型与数据拟合良好，但仍存在一些偏差。

为了解这些偏差的程度，比较累积分布函数会很有帮助。我们可以使用 make_cdf 来计算数据和模型的累积分布函数。

In [ ]:
cdf_family = pmf_family.make_cdf()
cdf_poisson = pmf_poisson.make_cdf()

In [ ]:
from thinkstats import two_cdf_plots

two_cdf_plots(cdf_poisson, cdf_family)
decorate(xlabel="Number of family members")

当我们比较累积分布函数时，差异不那么明显，但我们仍能看出分布的不同之处及其方式。概率质量函数倾向于强调细微差别——有时累积分布函数能更好地呈现整体趋势。

CDF 同样适用于连续数据。例如，我们再次查看出生体重的分布情况，该数据位于 NSFG 妊娠文件中。

In [ ]:
from nsfg import read_fem_preg

preg = read_fem_preg()
birth_weights = preg["totalwgt_lb"].dropna()

这是我们上一章中用于将正态模型拟合到数据的代码。

In [ ]:
from scipy.stats import trimboth
from thinkstats import make_normal_model

trimmed = trimboth(birth_weights, 0.01)
cdf_model = make_normal_model(trimmed)

In [ ]:
from empiricaldist import Cdf

cdf_birth_weight = Cdf.from_seq(birth_weights, name="sample")
two_cdf_plots(cdf_model, cdf_birth_weight, xlabel="Birth weight (pounds)")

正如我们在前一章所见，除了最轻婴儿的体重范围外，正态模型与数据拟合得很好。

在我看来，累积分布函数通常是数据与模型对比的最佳方式。但对于不熟悉累积分布函数的读者来说，还有另一种选择：概率密度函数。

## 6.2 Probability Density

我们将从正态分布的概率密度函数（PDF）开始，它根据给定的 mu 和 sigma 计算量值 xs 的密度。

In [ ]:
def normal_pdf(xs, mu, sigma):
    """Evaluates the normal probability density function."""
    z = (xs - mu) / sigma
    return np.exp(-(z**2) / 2) / sigma / np.sqrt(2 * np.pi)

对于 mu 和 sigma ，我们将使用修剪后出生体重的均值和标准差。

In [ ]:
m, s = np.mean(trimmed), np.std(trimmed)
m, s

现在我们将针对一系列体重值评估 normal_pdf 。

In [ ]:
low = m - 4 * s
high = m + 4 * s
qs = np.linspace(low, high, 201)
ps = normal_pdf(qs, m, s)

In [ ]:
plt.plot(qs, ps, label="normal model", ls=":", color="gray")
decorate(xlabel="Birth weight (pounds)", ylabel="Density")

结果看起来像一个钟形曲线，这是正态分布的典型特征。

当我们计算 normal_pdf 时，得到的结果是一个概率密度。例如，这是在均值处计算的密度函数值，该点也是密度最高的位置。

In [ ]:
normal_pdf(m, m, s)

概率密度本身意义不大——最重要的是，它并非概率。若声称随机选择的出生体重恰好等于 m 的概率为 32%，这是错误的。事实上，出生体重完全精确地等于 m ——或任何其他特定值——的概率为零。

然而，我们可以通过计算曲线下的面积，利用概率密度来计算结果落在两个值之间的区间内的概率。

我们可以使用 normal_pdf 函数来实现，但更方便的是使用 NormalPdf 类，它定义在 thinkstats 模块中。以下是如何创建一个 NormalPdf 对象，使其具有与 NSFG 数据集中出生体重相同的均值和标准差。

In [ ]:
from thinkstats import NormalPdf

pdf_model = NormalPdf(m, s, name="normal model")
pdf_model

如果我们像调用函数一样调用这个对象，它会计算正态概率密度函数。

In [ ]:
pdf_model(m)

现在，为了计算概率密度函数下的面积，我们可以使用以下函数，该函数接受一个 NormalPdf 对象以及区间的边界 low 和 high 。它在 low 和 high 之间等间距的数值上评估正态概率密度函数，并使用 SciPy 函数 simpson 来估计曲线下的面积（ simpson 之所以如此命名，是因为它使用了一种称为辛普森方法的算法）。

In [ ]:
from scipy.integrate import simpson


def area_under(pdf, low, high):
    qs = np.linspace(low, high, 501)
    ps = pdf(qs)
    return simpson(y=ps, x=qs)

如果我们计算从图中最低点到最高点曲线下的面积，结果接近于 1。

In [ ]:
area_under(pdf_model, 2, 12)

如果我们把区间从负无穷延伸到正无穷，那么总面积恰好为 1。

如果我们从 0 开始——或者任何远低于均值的数值——我们可以计算出出生体重小于或等于 8.5 磅的比例。

In [ ]:
area_under(pdf_model, 0, 8.5)

你可能还记得，“小于或等于给定值的比例”是累积分布函数的定义。因此，我们可以使用正态分布的累积分布函数来计算相同的结果。

In [ ]:
from scipy.stats import norm

norm.cdf(8.5, m, s)

同样地，我们可以利用密度曲线下的面积来计算出生体重在 6 到 8 磅之间的比例。

In [ ]:
area_under(pdf_model, 6, 8)

或者，我们可以使用累积分布函数来计算小于 8 的比例，然后减去小于 6 的比例，从而得到相同的结果。

In [ ]:
norm.cdf(8, m, s) - norm.cdf(6, m, s)

因此，累积分布函数是概率密度函数曲线下的面积。如果你了解微积分，另一种表达方式是：累积分布函数是概率密度函数的积分。反之，概率密度函数是累积分布函数的导数。

## 6.3 The Exponential PDF

要理解概率密度，看另一个例子或许有帮助。在上一章中，我们使用指数分布来模拟冰球比赛中第一个进球的时间。我们使用了以下函数来计算指数分布的累积分布函数，其中 lam 是单位时间内的进球率。

In [ ]:
def exponential_cdf(x, lam):
    """Compute the exponential CDF.

    x: float or sequence of floats
    lam: rate parameter

    returns: float or NumPy array of cumulative probability
    """
    return 1 - np.exp(-lam * x)

我们可以这样计算指数分布的概率密度函数。

In [ ]:
def exponential_pdf(x, lam):
    """Evaluates the exponential PDF.

    x: float or sequence of floats
    lam: rate parameter

    returns: float or NumPy array of probability density
    """
    return lam * np.exp(-lam * x)

thinkstats 提供了一个 ExponentialPdf 对象，该对象使用此函数来计算指数概率密度函数。我们可以用它来表示每场比赛进 6 球的指数分布。

In [ ]:
from thinkstats import ExponentialPdf

lam = 6
pdf_expo = ExponentialPdf(lam, name="exponential model")
pdf_expo

ExponentialPdf 提供了一个 plot 函数，我们可以用它来绘制概率密度函数图——请注意，这里的时间单位是游戏次数，而非前一章所用的秒数。



In [ ]:
qs = np.linspace(0, 1.5, 201)
pdf_expo.plot(qs, ls=":", color="gray")
decorate(xlabel="Time (games)", ylabel="Density")

观察 y 轴，你可能会注意到其中一些密度值大于 1，这提醒我们概率密度并非概率本身。但密度曲线下的面积代表概率，因此该面积绝不应超过 1。

> - _在 x=0.2（即全场比赛 20% 的时间，约 12 分钟）处的概率密度高于 x=0.4（全场 40% 的时间，约 24 分钟）处_
> - _这意味着： 在观察大量的进球数据时，你会发现两个进球之间相隔 12 分钟的情况（频率或可能性浓度），要比相隔 24 分钟的情况多得多_
> - _x 轴不是比赛时钟： 这个图表的 x 轴代表的是两次进球之间的时间长度（Interval），而不是“比赛进行到了第几分钟”_
> - _恒定进球率： 该分布背后的物理模型假设：进球在比赛的任何一秒钟发生的概率都是完全相同的（恒定速率 λ）_
> - _结论： 无论比赛是在第 1 分钟还是第 59 分钟，进球的难度（概率）在模型中是一模一样的_
> - _逻辑角度（维持“不进球”很难）： 要产生一个“40% 场次（24 分钟）的长间隔”，需要在这连续的 1440 秒内，每一秒钟都不能进球。只要其中任何一秒“破功”进了球，这个大间隔就会断裂成两个更小的间隔。因此，在随机发生的事件中，长时间维持“什么都不发生”是非常罕见的_

如果我们计算从 0 到 1.5 场比赛的曲线下面积，可以确认结果接近 1。

In [ ]:
area_under(pdf_expo, 0, 1.5)

如果我们大幅延长区间，结果会略大于 1，但这只是因为我们采用了数值近似计算面积。从数学角度严格证明，通过指数分布的累积分布函数可以确认，其面积恰好等于 1。

In [ ]:
from thinkstats import exponential_cdf

exponential_cdf(7, lam)

我们可以利用密度曲线下的面积来计算任意时间段内进球的概率。例如，以下是 60 分钟比赛中第一分钟内进球的概率。

In [ ]:
area_under(pdf_expo, 0, 1 / 60)

我们可以使用指数累积分布函数来计算相同的结果。

In [ ]:
exponential_cdf(1 / 60, lam)

总而言之，如果我们评估一个概率密度函数，得到的结果是概率密度——而非概率。然而，如果我们计算概率密度函数下的面积，结果就是某个量落在某个区间内的概率。或者，我们也可以通过计算累积分布函数在区间起点和终点的值，并求其差值来得到相同的概率。

## 离散化 & 连续化

从数据和理论模型的视角来看，**离散化（Discretization）和连续化（具体体现为核密度估计 KDE）** 是统计分析中为了统一“语言”而进行的两种相反方向的转换过程。

- 离散化 (Discretize)：将连续模型转化为离散形式
  - 逻辑与实现：当我们拥有一个连续的数学模型（如正态分布的 PDF）时，该模型在任何精确点的概率都为零。为了将其与离散的观测数据进行对比，我们需要在模型曲线上选取一系列离散的点，并计算这些点上的 PDF 值
  - 归一化处理：通过将这些点对应的密度值进行归一化，使其总和为 1，从而产生一个包含“概率质量”的近似 PMF（概率质量函数）
  目的：这样做是为了让理论模型能够以“离散点对”的形式，与原始数据的 PMF 在相同的坐标轴上进行绘图和比较
- 连续化 (Continuization/KDE)：将离散数据转化为连续形式
  - 逻辑与实现：核密度估计（KDE）概率密度核（通常是正态分布），然后将所有核叠加起来
  - 平滑噪声：原始数据的 PMF 往往包含很多随机的尖峰和“噪音”，而通过 KDE 得到的估计 PDF 是平滑的，更能反映出数据背后潜藏的连续物理规律
  - 目的：它使得我们能够将样本数据转化成密度曲线，从而与完美的理论 PDF 曲线（如理想正态分布曲线）直接重叠对比，以评估模型的拟合效果

这两者背后的根本目的是为了解决**数据与模型之间“单位与量级不匹配”**的问题：

- 离散化是让理论模型迁就观测数据的离散特性
- 连续化 (KDE) 是让观测数据还原其底层连续的物理本质，并消除由于样本量有限而产生的随机噪声

在实际应用中，作者认为虽然这两种方法可以辅助可视化，但**直接比较累积分布函数（CDF）**通常是评估数据与模型一致性的最稳健方法，因为 CDF 能够避开概率密度带来的抽象困扰，且无需进行复杂的离散或连续转换

## 6.6 The Distribution Framework

至此，我们已经掌握了一整套表示分布的方法：概率质量函数（PMF）、累积分布函数（CDF）和概率密度函数（PDF）。下图展示了这些表示形式以及它们之间的转换关系。例如，如果我们有一个 Pmf ，我们可以使用 cumsum 函数计算概率的累积和，从而得到一个表示相同分布的 Cdf

![alt](./distribution_framework.png)


为了展示这些转换过程，我们将使用一个新的数据集，根据描述，该数据集“包含了澳大利亚布里斯班一家医院在 24 小时内出生的 44 名婴儿的出生时间、性别和出生体重”。下载数据的说明位于本章的笔记本中。

> 数据背后的故事：1997 年 12 月 18 日，在澳大利亚昆士兰州布里斯班的玛特母亲医院，24 小时内诞生了 44 名婴儿——创下了新纪录。《星期日邮报》记录了这 44 名婴儿的出生时间、性别以及以克为单位的出生体重。

In [ ]:
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/babyboom.dat")

In [ ]:
from thinkstats import read_baby_boom

boom = read_baby_boom()
boom.head()

minutes 列记录了“每个婴儿出生距离午夜的分钟数”。因此，我们可以使用 diff 方法来计算每个连续出生之间的时间间隔。



In [ ]:
diffs = boom["minutes"].diff().dropna()
diffs.head()

如果婴儿出生在一天中的任何一分钟内概率相等，我们预期这些间隔时间会遵循指数分布。实际上，这一假设并非完全准确，但指数分布仍可能是数据的一个良好模型。

为了找出答案，我们将首先创建一个 Pmf ，用于表示间隔的分布。

In [ ]:
from empiricaldist import Pmf
from thinkstats import decorate

pmf_diffs = Pmf.from_seq(diffs, name="diffs")
pmf_diffs.bar(width=1)
decorate(xlabel="Interval (minutes)", ylabel="PMF")

然后我们可以使用 make_cdf 来计算累积概率，并将其存储在一个 Cdf 对象中。

In [ ]:
cdf_diffs = pmf_diffs.make_cdf()
cdf_diffs.step()

decorate(xlabel="Interval (minutes)", ylabel="CDF")

Pmf 和 Cdf 在某种意义上等价，因为只要给定其中一个，我们就能计算出另一个。为了演示这一点，我们将使用 make_pmf 方法，该方法计算 Cdf 中连续概率之间的差值，并返回一个 Pmf 。

In [ ]:
pmf_diffs2 = cdf_diffs.make_pmf()

结果应与原始 Pmf 完全相同，但可能存在微小的浮点误差。我们可以使用 allclose 来检查结果是否接近原始 Pmf 。

In [ ]:
import numpy as np

is_same = np.allclose(pmf_diffs, pmf_diffs2)
print(f"pmf_diffs 和 pmf_diffs2 是否相同: {is_same}")

从 Pmf 中，我们可以通过调用 gaussian_kde ，并将 Pmf 中的概率作为权重，来估计一个密度函数。

In [ ]:
pmf_diffs.head(5)

In [ ]:
from scipy.stats import gaussian_kde

kde = gaussian_kde(pmf_diffs.qs, weights=pmf_diffs.ps)

为了绘制结果，我们可以使用 kde 来创建一个 Pdf 对象，并调用 plot 方法。

In [ ]:
from thinkstats import Pdf

domain = np.min(pmf_diffs.qs), np.max(pmf_diffs.qs)
kde_diffs = Pdf(kde, domain=domain, name="estimated density")

kde_diffs.plot(ls=":", color="gray")
decorate(xlabel="Interval (minutes)", ylabel="Density")

为了判断估计的密度是否遵循指数模型，我们可以创建一个与数据均值相同的 ExponentialCdf 。

In [ ]:
from thinkstats import ExponentialCdf

m = diffs.mean()
lam = 1 / m
cdf_model = ExponentialCdf(lam, name="exponential CDF")

与数据的累积分布函数相比，其形态如下所示。

In [ ]:
cdf_model.plot(ls=":", color="gray")
cdf_diffs.step()

decorate(xlabel="Interval (minutes)", ylabel="CDF")

指数模型很好地拟合了数据的累积分布函数。

给定一个 ExponentialCdf ，我们可以使用 make_cdf 来离散化 CDF——也就是说，通过在一系列等间距的量上评估 CDF 来制作一个离散近似。

In [ ]:
qs = np.linspace(0, 160)
discrete_cdf_model = cdf_model.make_cdf(qs)
discrete_cdf_model.step(color="gray")

decorate(xlabel="Interval (minutes)", ylabel="CDF")

最后，为了从离散累积分布函数过渡到连续累积分布函数，我们可以在阶梯之间进行插值，这正是我们使用 plot 方法而非 step 方法时所观察到的现象。

In [ ]:
discrete_cdf_model.plot(color="gray")

decorate(xlabel="Interval (minutes)", ylabel="CDF")

最后，概率密度函数是连续累积分布函数的导数，而累积分布函数是概率密度函数的积分。

为了演示，我们可以使用 SymPy 来定义指数分布的累积分布函数并计算其导数。

In [ ]:
import sympy as sp

x = sp.Symbol("x", real=True, positive=True)
λ = sp.Symbol("λ", real=True, positive=True)

cdf = 1 - sp.exp(-λ * x)
cdf

In [ ]:
pdf = sp.diff(cdf, x)
pdf

如果我们对结果进行积分，就能重新得到累积分布函数——尽管在这个过程中我们失去了积分常数。

In [ ]:
sp.integrate(pdf, x)

此示例展示了我们如何使用 Pmf 、 Cdf 和 Pdf 对象来表示概率质量函数、累积分布函数和概率密度函数，并演示了从一种形式转换为另一种形式的过程。

## 6.8 Exercise

### 6.8.1 Exercise 6.1

在世界杯足球赛中，假设第一个进球的时间很好地服从指数分布，速率为每场比赛 lam=2.5 个进球。创建一个 ExponentialPdf 来表示这个分布，并使用 area_under 来计算第一个进球时间小于半场比赛的概率。然后使用 ExponentialCdf 来计算相同的概率，并检查结果是否一致。

使用 ExponentialPdf 计算第一个进球发生在比赛下半场的概率。然后使用 ExponentialCdf 计算相同概率，并验证结果是否一致。

**下面使用 scipy 包来计算. 这里不采用作者提供的包, 目的是为了学习了解 python 生态下数据科学常用包.**

In [ ]:
import numpy as np
from scipy.integrate import simpson
from scipy.stats import expon

1. 参数设定

In [ ]:
# 每场比赛平均进球数 (单位: 球/场)
lam = 2.5
# 进球速率. 表示平均多少场可以进一个球 (单位: 场). 对应到 scipy 就是 scale
scale = 1 / lam

> `lam` 和 `scale` 的关系: 
> - $lam=\frac{2.5球}{1场}$
> - $scale=\frac{1}{lam}=\frac{1球}{\frac{2.5球}{1场}}=1球 \times \frac{1场}{2.5球}=\frac{1}{2.5}$场=0.4场
> - 表示进 1球 需要 0.4 场的意思

> 我们可以用速度举例来更加直观的理解它们的关系. 假设一辆汽车的速度是 120km/小时
> - $lam=\frac{120km}{1h}$
> - $scale=\frac{1km}{\frac{120km}{1h}}=\frac{1h}{120}$=0.0083
> - 表示 1km 需要 0.0083小时 的意思

2. 计算 "第一个进球在半场前 (x < 0.5)" 的概率

In [ ]:
# 使用 PDF (面积法)
# 逻辑: 生成 0 到 0.5 之间的点, 计算对应的密度值, 然后利用辛普森法则求积分

# 生成 0 到 0.5 之间的点
qs_1 = np.linspace(0, 0.5, 501)
# 获取 PDF 密度值
ps_1 = expon.pdf(qs_1, scale=scale)
# 计算面积
prob1_pdf = simpson(ps_1, qs_1)

print(f"第一个进球在半场前的概率(PDF 面积法): {prob1_pdf:.4f}")
# 使用 CDF (累积概率法)
# 逻辑: cdf(x) 直接返回 P(X <= x) 的概率
prob1_cdf = expon.cdf(0.5, scale=scale)

print(f"第一个进球在半场前的概率(CDF 累积概率法): {prob1_cdf:.4f}")

3. 计算"第一个进球在下半场 (0.5 < x < 1.0)"的概率

In [ ]:
# 使用 PDF (面积法)
# 逻辑: 生成 0.5 到 1.0 之间的点, 计算对应的密度值, 然后利用辛普森法则求积分

# 生成 0.5 到 1.0 之间的点
qs_2 = np.linspace(0.5, 1.0, 501)
# 获取 PDF 密度值
ps_2 = expon.pdf(qs_2, scale=scale)
# 计算面积
prob2_pdf = simpson(ps_2, qs_2)

print(f"第一个进球在下半场的概率(PDF 面积法): {prob2_pdf:.4f}")

# 使用 CDF (累积概率法)
# 逻辑: 通过 CDF(1.0)−CDF(0.5) 计算落在该区间的概率
prob2_cdf = expon.cdf(1.0, scale=scale) - expon.cdf(0.5, scale=scale)
print(f"第一个进球在下半场的概率(CDF 累积概率法): {prob2_cdf:.4f}")

### 6.8.2 Exercise 6.2

要加入蓝人乐团，你必须是一名身高在 5 英尺 10 英寸到 6 英尺 1 英寸之间的男性，这大约是 178 到 185 厘米。让我们看看美国成年男性中有多大比例符合这一要求。

BRFSS 男性参与者的身高数据很好地符合正态分布模型，其平均值为 178 厘米，标准差为 7 厘米。

In [ ]:
from thinkstats import read_brfss

brfss = read_brfss()
male = brfss.query("sex == 1")
heights = male["htm3"].dropna()

In [ ]:
from scipy.stats import trimboth

trimmed = trimboth(heights, 0.01)
m, s = np.mean(trimmed), np.std(trimmed)

print(f"平均身高: {m:.2f} 厘米, 标准差: {s:.2f} 厘米")

这是一个 NormalCdf 对象，代表一个与修剪后数据具有相同均值和标准差的正态分布。

In [ ]:
from thinkstats import NormalCdf

cdf_normal_model = NormalCdf(m, s, name="normal model")

以下是数据累积分布函数的对比情况。

In [ ]:
cdf_height = Cdf.from_seq(heights, name="data")
cdf_normal_model.plot(ls=":", color="gray")
cdf_height.step()

xlim = [140, 210]
decorate(xlabel="Height (cm)", ylabel="CDF", xlim=xlim)

第一题: 使用 gaussian_kde 来创建一个近似男性身高概率密度函数的 Pdf 。提示：研究 bw_method 参数，该参数可用于控制估计密度的平滑度。绘制估计密度图，并将其与均值为 m 、标准差为 s 的 NormalPdf 进行比较。

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm, gaussian_kde, trimboth
from scipy.integrate import simpson

# 生成 x 轴范围: 均值上下 4 个标准差
qs = np.linspace(m - 4 * s, m + 4 * s, 501)

# 1. 理论正态分布
ps_normal = norm.pdf(qs, loc=m, scale=s)

# 2. 核密度估计
# 注意: 此处假设 heights 是原始男性身高数组
kde = gaussian_kde(heights)
ps_kde = kde(qs)

# 3. 绘图对比
plt.plot(qs, ps_normal, label="Normal Distribution", color="blue")
plt.plot(qs, ps_kde, label="KDE", color="red")
plt.xlabel("Height (cm)")
plt.ylabel("Density")
plt.legend()
plt.show()

第二题: 使用 NormalPdf 和 area_under 计算正态模型中身高在 178 至 185 厘米之间的人口比例。使用 NormalCdf 计算相同的比例，并验证结果是否一致。最后，利用数据的经验 Cdf 来查看数据集中处于相同身高范围的人口比例。

In [ ]:
# 方式A: 使用 PDF 面积法

# 1. 在目标区间生成数据
qs_range = np.linspace(178, 185, 501)
ps_range = norm.pdf(qs_range, loc=m, scale=s)

# 2. 使用 simpson 计算面积
prob_pdf_area = simpson(y=ps_range, x=qs_range)
print(f"使用 PDF 面积法计算的身高在 178 至 185 厘米之间的人口比例: {prob_pdf_area:.4f}")

# 方式B: 使用 CDF 差值法

# P(178 <= X <= 185) = P(X <= 185) - P(X <= 178)
prob_cdf_diff = norm.cdf(185, loc=m, scale=s) - norm.cdf(178, loc=m, scale=s)
print(f"使用 CDF 差值法计算的身高在 178 至 185 厘米之间的人口比例: {prob_cdf_diff:.4f}")